# 🤗 HuggingFace Transformer Fine-Tuning for Football Prediction

Fine-tune pre-trained transformers on football match data.

**Features:**
- Load Podos or similar pre-trained models
- Custom football prediction head
- Export to ONNX for fast inference

In [ ]:
!pip install -q torch transformers huggingface_hub onnx onnxruntime

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')

## 1. Football Transformer Model

In [ ]:
class FootballTransformer(nn.Module):
    '''Transformer model for football match prediction'''
    def __init__(self, input_dim=23, d_model=128, nhead=4, num_layers=3, num_classes=3):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.classifier = nn.Sequential(
            nn.Linear(d_model, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )
    
    def forward(self, x):
        x = self.input_proj(x).unsqueeze(1)  # Add sequence dimension
        x = self.transformer(x)
        x = x.mean(dim=1)  # Pool
        return self.classifier(x)

model = FootballTransformer().to(DEVICE)
print(f'Parameters: {sum(p.numel() for p in model.parameters()):,}')

## 2. Prepare Data

In [ ]:
# Load or create sample data
try:
    df = pd.read_csv('/kaggle/input/international-football-results-from-1872-to-2017/results.csv')
except:
    np.random.seed(42)
    df = pd.DataFrame({
        'home_score': np.random.randint(0, 5, 5000),
        'away_score': np.random.randint(0, 5, 5000)
    })

# Create features (expand as needed)
features = np.random.randn(len(df), 23).astype(np.float32)  # Placeholder features
labels = np.where(df['home_score'] > df['away_score'], 0,
                  np.where(df['home_score'] < df['away_score'], 2, 1))

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)
print(f'Train: {len(X_train)}, Test: {len(X_test)}')

## 3. Train

In [ ]:
class MatchDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)
    def __len__(self): return len(self.y)
    def __getitem__(self, i): return self.X[i], self.y[i]

train_loader = DataLoader(MatchDataset(X_train, y_train), batch_size=64, shuffle=True)
test_loader = DataLoader(MatchDataset(X_test, y_test), batch_size=64)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

for epoch in range(30):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
        optimizer.zero_grad()
        loss = criterion(model(X_batch), y_batch)
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        model.eval()
        correct = 0
        with torch.no_grad():
            for X_b, y_b in test_loader:
                correct += (model(X_b.to(DEVICE)).argmax(1).cpu() == y_b).sum().item()
        print(f'Epoch {epoch+1}: Acc = {correct/len(X_test):.4f}')

## 4. Export to ONNX

In [ ]:
model.eval()
dummy = torch.randn(1, 23).to(DEVICE)

torch.onnx.export(
    model, dummy, 'football_transformer.onnx',
    input_names=['features'],
    output_names=['logits'],
    dynamic_axes={'features': {0: 'batch'}, 'logits': {0: 'batch'}}
)

# Also save PyTorch weights
torch.save(model.state_dict(), 'football_transformer.pt')
print('Exported football_transformer.onnx and .pt')

## 📥 Download

Place files in `models/trained/`:
- `football_transformer.onnx`
- `football_transformer.pt`